Ref: https://milvus.io/docs/search.md

In [ ]:
import random
import random
import numpy as np
from pprint import pprint

#from milvus import Milvus

In [40]:
from pymilvus import connections,CollectionSchema, FieldSchema, DataType,utility

In [16]:
#Connects to a server:
connections.connect(alias="default", host="localhost", port="19530",user="root",password="Milvus")

In [22]:
#Preparing schema
fields = [
    FieldSchema(name="book_id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="book_name", dtype=DataType.VARCHAR, max_length=200,default_value="Unknown"),
    FieldSchema(name="word_count", dtype=DataType.INT64, default_value=9999),
    FieldSchema(name="book_intro", dtype=DataType.FLOAT_VECTOR, dim=2)
]
schema = CollectionSchema(fields,description="Test book search",enable_dynamic_field=True) # No.of fields in collection=64

collection_name = "book"  #len(collection_name)= max 255 characters

In [24]:
#Creating collection with schema
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2  # Number of shards in a collection=16
    )

In [33]:
#Preparing data
data = [
  [i for i in range(2000)],
  [str(i) for i in range(2000)],
  [i for i in range(10000, 12000)],
  [[random.random() for _ in range(2)] for _ in range(2000)]
]

#data.append([str("dy"*i) for i in range(2000)])

In [38]:
#Insert the data to the collection/Milvus
from pymilvus import Collection
collection = Collection("book")      # Get an existing collection.
mr = collection.insert(data)
# After final entity is inserted, it is best to call flush to have no growing segments left in memory

In [39]:
# preparing index parameters
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}

In [41]:
#Build the index by specifying the vector field name and index parameters.
# Get an existing collection.
collection = Collection("book")      
collection.create_index(
  field_name="book_intro", 
  index_params=index_params
)

utility.index_building_progress("book")

{'total_rows': 10000, 'indexed_rows': 10000}

In [47]:
#Conduct a Vector Similarity Search
# 1. Load collection
collection = Collection("book")      # Get an existing collection.
collection.load()
# 2. Prepare search parameters
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}
# 3. Conduct a vector search
results = collection.search(
    data=[[0.7, 0.71]], 
    anns_field="book_intro", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=10,
    expr=None,
    # set the names of the fields you want to 
    # retrieve from the search result.
    output_fields=['title'],
    consistency_level="Strong"
)


In [ ]:
connections.disconnect("default")